In [1]:
from pydantic import BaseModel, validator, Field
from typing import Optional, Union
from pyfuseki import FusekiUpdate
import json
from datetime import date, datetime
from api.src.function.bibframe.Work.graph import MakeGraph
import httpx

import pysolr  

In [1]:
from rdflib import Graph
from api.src.schemas.bibframe.work import Work
from api.src.function.loc.bibframe.getType import GetType
from pydantic import BaseModel

In [13]:
from api.src.schemas.metadata.bibframe.adminMetadata import AdminMetadata
from api.src.schemas.metadata.bibframe.classification import Classification
from api.src.schemas.metadata.bibframe.content import Content
from api.src.schemas.metadata.bibframe.contribution import Contribution

class Work(BaseModel):
    adminMetadata: AdminMetadata
    type: list[str]
    classification: Classification
    content: Content
    contribution: list[Contribution]
    # genreForm: Optional[list[GenreForm]]
    # language: list[Language]
    # note: Optional[str]
    # summary: Optional[str]
    # title: Title
    # subject: list[Topic]
    # illustrativeContent: Optional[list[Illustration]]
    # supplementaryContent: Optional[list[SupplementaryContent]]
    # expressionOf: Optional[str]
    # hasInstance: Optional[str]

obj = {
    'adminMetadata': {
        'identifiedBy': [ 
            {
                "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
            "value": "22600263"
            }
        ]
    },
    "type": [
        "Work"
    ]
}
# r = Work(**obj)
# r

In [3]:
graph = Graph()
uri = 'http://id.loc.gov/resources/works/22600263'
graph.parse(f'{uri}.rdf')
# g.serialize('work.ttl', format='ttl')

<Graph identifier=N876b20f804e4464cb04abee09b35e609 (<class 'rdflib.graph.Graph'>)>

In [26]:
GetType(graph, uri)

['Work', 'Text', 'Monograph']

In [14]:
def GetContent(graph, uri, obj):

  labels = { 'txt': 'Texto'}

  q = f"""PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
              PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
              PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
              SELECT ?type ?agent ?role
                WHERE {{ 
                  <{uri}> bf:contribution ?contribution .
              ?contribution rdf:type ?type .
              ?contribution bf:agent ?agent .
              ?contribution bf:role ?role
                  }}"""
  response = graph.query(q)
  bindings = response.bindings
  # if len(bindings) > 0:
  #   binding = bindings[0]
  #   valeu = binding.get('content').toPython()
  #   code = valeu.split('/')[-1]
  #   content = {
  #     'value': valeu,
  #     'label': labels[code]
  #     }
  #   obj['content'] = content
    
  return bindings

bindings = GetContent(graph, uri, obj)
bindings

[{rdflib.term.Variable('agent'): rdflib.term.URIRef('http://id.loc.gov/rwo/agents/n86870523'), rdflib.term.Variable('role'): rdflib.term.URIRef('http://id.loc.gov/vocabulary/relators/aut'), rdflib.term.Variable('type'): rdflib.term.URIRef('http://id.loc.gov/ontologies/bibframe/Contribution')},
 {rdflib.term.Variable('agent'): rdflib.term.URIRef('http://id.loc.gov/rwo/agents/n86870523'), rdflib.term.Variable('role'): rdflib.term.URIRef('http://id.loc.gov/vocabulary/relators/aut'), rdflib.term.Variable('type'): rdflib.term.URIRef('http://id.loc.gov/ontologies/bflc/PrimaryContribution')}]

In [12]:
from api.src.function.loc.bibframe.getClassification import GetClassification


def ParserWork(graph, uri):
    identifier = uri.split("/")[-1]
    types = GetType(graph, uri)

    obj = {
    'adminMetadata': {
        'generationProcess': {
            'label': 'BiblioKeia'
        },
        'identifiedBy': [ 
            {
                "type": "Local",
            "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
            "value": identifier
            }
        ]
    },
    "type": types
    }
    # Classification
    obj = GetClassification(graph, uri, obj)
    # Content
    obj = GetContent(graph, uri, obj)

    


    response = Work(**obj)
    return response

response = ParserWork(graph, uri)
response.dict()
    

{'adminMetadata': {'encodingLevel': 'menclvl:f',
  'assigner': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-06-02',
  'descriptionConventions': 'http://id.loc.gov/vocabulary/descriptionConventions/isbd',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': {'label': 'BiblioKeia',
   'generationDate': '2023-06-02T16:16:41'},
  'identifiedBy': [{'type': 'Local',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': '22600263'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'type': ['Work', 'Text', 'Monograph'],
 'classification': {'type': 'ClassificationDdc',
  'classificationPortion': '338.9001',
  'itemPortion': None,
  'edition': 'full'},
 'content': {'label': 'Texto',
  'value': 'http://id.loc.gov/vocabulary/contentTypes/txt'}}

In [12]:
authority

NameError: name 'authority' is not defined

In [27]:


  # tipo = r.bindings[0].get('type').split("#")[1]
  
  return r

obj = GetClassification(graph, uri, {})
obj

{'classification': {'classificationPortion': '338.9001',
  'itemPortion': None,
  'edition': 'full'}}

In [24]:
response.get('itemPortion')

In [4]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

response = fuseki_update.run_sparql(graph)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [11]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = fuseki_update.run_sparql(d)
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

Exemple Work Loc

In [53]:
g = Graph()
g.parse('https://www.loc.gov/standards/mads/rdf/v1/examples/oca08651008.nt', format='nt')

<Graph identifier=N6e8f6b25d8384e1585946c4b1a9f8153 (<class 'rdflib.graph.Graph'>)>

In [54]:
g.serialize('EX_NAME.ttl', format='ttl')

<Graph identifier=N6e8f6b25d8384e1585946c4b1a9f8153 (<class 'rdflib.graph.Graph'>)>

GET WORK

In [45]:
response = httpx.get('http://localhost:3030/authorities?graph=https://bibliokeia.com/authorities/name/n80002329')
response

<Response [200 OK]>

In [46]:
g = Graph()
g.parse(response.content)
g.serialize('name.ttl', format='ttl')

<Graph identifier=Nc7fd2ec4e1e541edbd048c09e685ea53 (<class 'rdflib.graph.Graph'>)>

In [8]:
g.serialize('bk.rdf', format='xml')

<Graph identifier=Na2c213dac7fc4260b1e9a3ce33f14215 (<class 'rdflib.graph.Graph'>)>

In [51]:
for contribution in request.contribution:
    print(contribution.agent)

https://bibliokeia.com/authorities/name/n80002329


In [42]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = """PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH <{subject}> {{ 
                        <{subject}>  bf:contributionOf   <https://bibliokeia.com/resources/work/{id}> }} }} ; """
for i in request.subject:
    up = subjectOf.format(subject=i.value, id="bk-1")
    response = fuseki_authorities.run_sparql(up)
    print(response.convert())

{'statusCode': 200, 'message': 'Update succeeded'}


In [32]:
fuseki_authorities = FusekiUpdate('http://localhost:3030', 'authorities')
subjectOf = f"""PREFIX graph: <https://bibliokeia.com/authorities/topic/>
            PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

                INSERT DATA
                    {{ GRAPH graph:sh85017405 {{ 
                        graph:sh85017405  bf:subjectOf <https://bibliokeia.com/resources/work/bk-2> }} }} ; """
fuseki_authorities.run_sparql(subjectOf)

SOLR

In [40]:
request.genreForm

[GenreForm(label='Ficção', value='genreForms:gf2014026339')]

In [41]:
id = 'bk-1'
doc = {
    'id': id,
    'type': [i.replace("bf:", "") for i in request.type],
    'creationDate': request.adminMetadata.creationDate,
    'status': request.adminMetadata.status.label,
    'cdd': request.classification.classificationPortion,
    'cutter': request.classification.itemPortion,
    'content': [content.label for content in request.content],
    'contribution': [{'id':f'{id}!{contribution.agent.split("/")[-1]}', **contribution.dict()} for contribution in request.contribution],
    'genreForm': [genreForm.label for genreForm in request.genreForm ]
}
doc

{'id': 'bk-1',
 'type': ['Monograph', 'Text', 'Work'],
 'creationDate': '2023-04-20',
 'status': 'novo',
 'cdd': '869.3',
 'cutter': 'M513',
 'content': ['texto', 'Imagem'],
 'contribution': [{'id': 'bk-1!n80002329',
   'type': ['bf:Contribution', 'bflc:PrimaryContribution'],
   'agent': 'https://bibliokeia.com/authorities/name/n80002329',
   'label': 'Machado de Assis',
   'role': 'relators:aut'}],
 'genreForm': ['Ficção']}

In [42]:
solr = pysolr.Solr('http://localhost:8983/solr/acervo/', timeout=10)
r = solr.add([doc], commit=True)
r

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":128}}\n'

In [32]:
solr.delete(q="*:*", commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">44</int>\n</lst>\n</response>\n'